<a href="https://colab.research.google.com/github/RedaRafi/Face-Recognition-using-MTCNN/blob/main/Face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mtcnn


     |████████████████████████████████| 2.3MB 2.8MB/s 


In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import h5py
import cv2
import glob
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import os
from mtcnn.mtcnn import MTCNN

In [4]:
detector = MTCNN()

cap = cv2.VideoCapture(0)
numimg = 207
while cap.isOpened():
    _, img = cap.read() 
    
    result = detector.detect_faces(img)
     for faces in result:
        bounding_box = faces['box'] 
     #roi = test_image_gray[y:y+h, x:x+w]
     #save face in folder 
        dim = (128,128)
        if img != []:
            roi = img[ bounding_box[1]: bounding_box[1]+ bounding_box[3] ,bounding_box[0]:bounding_box[0]+bounding_box[2]] 
            if roi != [] :
                cv2.imwrite("C:/Users/"+str(numimg) + ".jpg",roi)
                cv2.rectangle(img,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,0,255),2)
    
    # Display the output
    cv2.imshow('img', img)
    numimg = numimg + 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 

In [ ]:
 def load_images_from_folder(pers ,folder):
    images = []
    personInPhoto = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            
            personInPhoto.append(pers)
            img = cv2.resize(img,(128,128))
            #img = np.reshape(img,[1,200,220,3])
            images.append(img)
    return (images , personInPhoto)

In [ ]:
PersonName0 ,pers0     = load_images_from_folder(0 ,"C:/Users/")
PersonName1,pers1       = load_images_from_folder(1 ,"C:/Users/")
X_test ,Y_test          =  load_images_from_folder(500 ,"C:/Users/")

# PersonName1    = np.array(imageAyoub[:])
PersonName0  = np.array(imageMouindo[:]) 
XT       = np.array(X_test[:])  

#data to train wait ok 
# X__train = np.concatenate((PersonName0 ,PersonName1))
X__train = PersonName0
# Y__train  = np.concatenate((np.array(pers0[:]) , np.array(pers1[:])))
Y__train = np.array(pers0[:])

In [ ]:
cv2.imshow("PersonName0",X__train[20])
cv2.waitKey(0)

In [ ]:
X__train  =  X__train  / 255

In [ ]:
cnn_model = Sequential()

cnn_model.add(Conv2D(64, 6, 6, input_shape = (128, 128, 3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(64, 5, 5, activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 128, activation = 'relu'))

cnn_model.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
cnn_model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr = 0.001), metrics = ['accuracy'])
  

In [ ]:
epochs = 10

#X_train,X_test,Y_train,Y_test=train_test_split(X_data_train_2,Y_data_output_2,test_size=0.3,random_state=random.seed())

history = cnn_model.fit(X__train, Y__train, batch_size = 30, epochs = epochs, verbose = 1)

In [ ]:
cnn_model.save('Face_detec_Model.h5')

In [ ]:
from keras.models import load_model 
cnn_model = load_model('Face_detec_Model.h5')


In [ ]:
predicted_classes = cnn_model.predict_classes(XT)

In [ ]:
detector = MTCNN()

cap = cv2.VideoCapture(0)
numimg = 207 
while cap.isOpened():
    _, img = cap.read() 
    
    result = detector.detect_faces(img)
    for faces in result:
        bounding_box = faces['box'] 
     #roi = test_image_gray[y:y+h, x:x+w]
     #save face in folder 
        dim = (128,128)
        roi = img[ bounding_box[1]: bounding_box[1]+ bounding_box[3] ,bounding_box[0]:bounding_box[0]+bounding_box[2]]  
        img1 = cv2.resize(roi,(128,128))
        img1 = np.reshape(img1,[1,128,128,3])
        img1 = img1 / 255
        
        predict = cnn_model.predict_classes(img1) 
        predict2 = cnn_model.predict_proba(img1)
        xx = np.round(predict2, 4)
        print(predict2)
        cv2.rectangle(img,
                      (bounding_box[0], bounding_box[1]),
                      (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                      (0,0,255),2)
        if predict == 0  :
            v = "Pers0 Details"
        elif predict == 1 :
            v = "Pers1 Detailsx"
        else :
            v = "unKown"
        cv2.putText(img,v,(bounding_box[0], bounding_box[1]), 1, 1,(0,255,0),2,cv2.LINE_AA)
     
    
    # Display the output
    cv2.imshow('img', img)
    numimg = numimg + 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 